In [1]:
import modin.pandas as pd
import os

We downloaded the data from October 2016 to March 2017 from citibike directly. https://citibikenyc.com/system-data
The downloaded data had the same schema with a different format for columns, so we renamed the columns to match the data extracted from the bigquery dataset. https://console.cloud.google.com/marketplace/details/city-of-new-york/nyc-citi-bike. The format of the gender column also changed between numeric and string. 

In [2]:
rename_dict = {
    'Trip Duration' : 'tripduration', 
    'Start Time' : 'starttime', 
    'Stop Time' : 'stoptime',
    'Start Station ID': 'start_station_id',
    'Start Station Name' : 'start_station_name',
    'Start Station Latitude' : 'start_station_latitude',
    'Start Station Longitude' : 'start_station_longitude',
    'End Station ID' : 'end_station_id', 
    'End Station Name' : 'end_station_name',
    'End Station Latitude' : 'end_station_latitude', 
    'End Station Longitude' : 'end_station_longitude',
    'Bike ID' : 'bikeid', 
    'User Type' : 'usertype',
    'Birth Year' : 'birth_year',
    'Gender' : 'gender'
}

In [3]:
fix_gender_format = {
    0: "unknown" ,
    1: "male",
    2:"female"
}

In [4]:
trips = None
for file in os.listdir("raw"):
    if ("DS_Store" in file):
        continue
    # print(file)
    month_trips = pd.read_csv("raw/" + file).sample(frac=0.01)
    month_trips.rename(columns=rename_dict,
          inplace=True)
    month_trips['gender'] = month_trips['gender'].apply(lambda t: fix_gender_format[t] )

    print(month_trips.gender.unique())
    if trips is None:
        trips = month_trips
    else:
        # print(month_trips.columns)
        trips = pd.concat([trips, month_trips])

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Perhaps you already have a cluster running?
Hosting the HTTP server on port 59956 instead


['female' 'male' 'unknown']
['male' 'unknown' 'female']
['male' 'female' 'unknown']
['male' 'female' 'unknown']
['female' 'male' 'unknown']
['male' 'female' 'unknown']


In [5]:
# load the data extracted from bigquery
bq_trips = pd.read_csv("bigquery-citibike-trips.csv")
# useless/empty column from the bigquery format
bq_trips = bq_trips.drop('customer_plan', axis=1)
print(bq_trips.columns)
fixed_trips = pd.concat([trips, bq_trips])

Index(['tripduration', 'starttime', 'stoptime', 'start_station_id',
       'start_station_name', 'start_station_latitude',
       'start_station_longitude', 'end_station_id', 'end_station_name',
       'end_station_latitude', 'end_station_longitude', 'bikeid', 'usertype',
       'birth_year', 'gender'],
      dtype='object')


In [6]:
# double checking the gender column was fixed
bq_trips.gender.unique()

array(['unknown', 'female', 'male'], dtype=object)

In [7]:
# Finally write out the cleaned up dataset
fixed_trips.to_csv("fixed-citibike-trips.csv", index=False)

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.
